Panel is not installed by default in the Pyodide distribution that JupyterLite is built on, therefore we must install it manually:

In [ ]:
import piplite
await piplite.install(['panel', 'pyodide-http', 'altair'])

Once installed we can activate the `pn.extension` as normal.

In [ ]:
import panel as pn
pn.extension('vega')

Panel components will now work just like in a regular notebook environment.

In [ ]:
w = pn.widgets.FloatSlider(start=0, end=3.14)

pn.Row(w, pn.bind(pn.pane.Str, w))

Even complex examples relying on custom extensions (e.g. `'vega'`) will work:

In [ ]:
import altair as alt
import pandas as pd

df = pd.read_json("https://raw.githubusercontent.com/vega/vega/master/docs/data/penguins.json")

brush = alt.selection_interval(name='brush')  # selection of type "interval"

chart = alt.Chart(df).mark_point().encode(
    x=alt.X('Beak Length (mm):Q', scale=alt.Scale(zero=False)),
    y=alt.Y('Beak Depth (mm):Q', scale=alt.Scale(zero=False)),
    color=alt.condition(brush, 'Species:N', alt.value('lightgray'))
).properties(
    width=250,
    height=250
).add_params(
    brush
)

vega_pane = pn.pane.Vega(chart, debounce=10)

def filtered_table(selection):
    if selection is None:
        return '## No selection'
    query = ' & '.join(
        f'{crange[0]:.3f} <= `{col}` <= {crange[1]:.3f}'
        for col, crange in selection.items()
    )
    return pn.pane.DataFrame(df.query(query), width=600)

pn.Column(
    'Select points on the plot and watch the linked table update.',
    sizing_mode='stretch_width'
).servable()

pn.Row(
    vega_pane,
    pn.Column(
        pn.bind(filtered_table, vega_pane.selection.param.brush),
        scroll=True, width=650, height=300
    )
)

The contents of Panelite are updated on every release, however Jupyterlite caches old edits to files. If you want to reset the storage and **delete all modifications you made to the contents** the click the button below.

In [ ]:
from IPython.display import display, HTML
display(HTML("""
<button type="button" id="button_for_indexeddb">Clear JupyterLite local storage</button>
<script>
window.button_for_indexeddb.onclick = function(e) {
    window.indexedDB.open('JupyterLite Storage').onsuccess = function(e) {
        // There are also other tables that I'm not clearing:
        // "counters", "settings", "local-storage-detect-blob-support"
        let tables = ["checkpoints", "files"];

        let db = e.target.result;
        let t = db.transaction(tables, "readwrite");

        function clearTable(tablename) {
            let st = t.objectStore(tablename);
            st.count().onsuccess = function(e) {
                console.log("Deleting " + e.target.result + " entries from " + tablename + "...");
                st.clear().onsuccess = function(e) {
                    console.log(tablename + " is cleared!");
                }
            }
        }

        for (let tablename of tables) {
            clearTable(tablename);
        }
    }
};
</script>
"""))